## JPA Hibernate Mapping

Java Persistance API (JPA) adalah standar java untuk mapping antara object dan relational database. Mapping antara object java dan relational database maupun sebaliknya disebut Object Relational Mapping (ORM). JPA merupakan salah satu pendekatan untuk ORM. Dengan menggunakan JPA, developer mampu untuk memetakan, menyimpan, memperbarui bahkan mengambil data relational database ke dalam object java ataupun sebaliknya.

Gambaran dari JPA Hibernate Mapping adalah sebagai berikut:

<img style="float: center;" src="https://imgur.com/KbcD70o.png" alt="JPA Hibernate Mapping"
	title="JPA Hibernate Mapping" width="500" height="300" />

### One to One

<img style="float: center;" src="https://imgur.com/oziNhos.png" alt="JPA Hibernate Mapping One to One"
	title="JPA Hibernate Mapping One to One" width="500" height="300" />
    
Implementasi pada Student.java

```java
@Entity
@Table(name = "tbl_student")
public class Student{
	
	@Id
	@GeneratedValue(strategy = GenerationType.IDENTITY)
	private Long id;
	
	@OneToOne(fetch=FetchType.LAZY, 
			cascade = CascadeType.ALL, 
			mappedBy = "student"
			)
	@JsonIgnore
	private Contact contact;
}
```

Implementasi pada Contact.java

```java
@Entity
@Table(name = "tbl_contact")
public class Contact{

	@Id
	@GeneratedValue(strategy = GenerationType.IDENTITY)
	private Long id;
	
	@OneToOne(fetch=FetchType.LAZY,
			optional = false)
	@JoinColumn(name = "student_id", nullable = false, unique = true)
	@JsonIgnore
	private Student student;
}
```

### One to Many

<img style="float: center;" src="https://imgur.com/XG1G1EZ.png" alt="JPA Hibernate Mapping One to Many"
	title="JPA Hibernate Mapping One to Many" width="500" height="300" />

Implementasi pada Student.java

```java
@Entity
@Table(name = "tbl_student")
public class Student{
	@Id
	@GeneratedValue(strategy = GenerationType.IDENTITY)
	private Long id;
	
	@OneToMany(fetch=FetchType.LAZY, 
			cascade = CascadeType.ALL, 
			mappedBy = "student")
	private List<Assignment> assignments;
}
```

Implementasi pada Assignment.java

```java
@Entity
@Table(name = "tbl_assignment")
public class Assignment {
	@Id
	@GeneratedValue(strategy = GenerationType.IDENTITY)
	private Long id;
	
	@ManyToOne(fetch=FetchType.LAZY, optional = false)
	@JoinColumn(name = "student_id", nullable = false, unique = true)
	private Student student;	
}

```

### Many to Many

<img style="float: center;" src="https://imgur.com/meaSy9A.png" alt="JPA Hibernate Mapping Many to Many"
	title="JPA Hibernate Mapping Many to Many" width="500" height="300" />

Implementasi pada Student.java

```java
@Entity
@Table(name = "tbl_student")
public class Student{
	@Id
	@GeneratedValue(strategy = GenerationType.IDENTITY)
	private Long id;
	
	@ManyToMany(cascade = CascadeType.ALL, fetch = FetchType.LAZY)
	@JoinTable(name = "student_subject", 
	joinColumns = @JoinColumn(name="student_id", referencedColumnName = "id"), 
	inverseJoinColumns = @JoinColumn(name="subject_id", referencedColumnName = "id"))
	private Set<Subject> subjects;
}
```

Implementasi pada Subject.java

```java
@Entity
@Table(name = "tbl_subject")
public class Subject {
	@Id
	@GeneratedValue(strategy = GenerationType.IDENTITY)
	private Long id;
	
	@Column(nullable = false, length=100)
	private String name;
	
	@ManyToMany(mappedBy = "subjects")
	private Set<Student> student;
}
```

## CRUD Restful

Spring JPA mampu menyediakan fungsi-fungsi CRUD tanpa harus membuat kodenya dari awal. Hanya dengan extends *generic function*  CrudRepository<T,ID> interface seperti pada contoh berikut:

```java
@Repository
public interface StudentRepository extends CrudRepository<Student, Long>{}
```

Class ```StudentRepository``` telah berhasil meng-extend beberapa method di antaranya: ```findAll(), findById(id), delete(), save()```

\* *generic function adalah template class yang parameter maupun return value-nya bisa tipe data apapun*

## JPA Auditing

JPA auditing berguna untuk melakukan *tracking* maupun *logging* yang dilakukan pada sebuah entitas. *Tracking* maupun *logging* yang dimaksud seperti siapa dan kapan object pada suatu entitas dibuat atau diubah. 

Ada 2 class yang dibuat, yaitu:

Auditable.java

```java
@Getter(value = AccessLevel.PROTECTED)
@Setter(value = AccessLevel.PROTECTED)
@MappedSuperclass
@EntityListeners(AuditingEntityListener.class)
public abstract class Auditable<U> {
	
	@CreatedBy
	protected U createdBy;
	
	@CreatedDate
	@Temporal(TemporalType.TIMESTAMP)
	protected Date createdDate;
	
	@LastModifiedBy
	protected U modifiedBy;
	
	@LastModifiedDate
	@Temporal(TemporalType.TIMESTAMP)
	protected Date modifiedDate;
}
```

AuditableConfig.java

```java
@Configuration
@EnableJpaAuditing(auditorAwareRef = "auditorProvider")
public class AuditableConfig {
	@Bean
	public AuditorAware<String> auditorProvider(){
		return ()->Optional.ofNullable("Admin");
	}
}
```

Kemudian model tinggal extend ke class Auditable untuk bisa ditambahkan kolom audit yang telah dibuat.